# Langchain to Bedrock Migration

In [2]:
%pip install langchain langchain-aws boto3 --upgrade

Note: you may need to restart the kernel to use updated packages.


## Step 0: Prerequisites and setup

In [2]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid

from agent import create_agent_role, create_lambda_role
from agent import create_dynamodb, create_lambda, invoke_agent_helper

#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
# region, account_id
agent_name = 'my-agent-'+region+account_id + str(time.time()).replace('.','-')
agent_name

'my-agent-us-east-17168459174841719414885-1153114'

## Step 1: Create and edit a local `lambda_function.py` 

In [3]:
%%writefile lambda_function.py


import json

def run_python_code(python_string):
    # Python code must contain "result" variable
    
    try:
        if "result" in python_string:
            
            print("Running the following python string")
            print(python_string)
            exec(python_string, globals())
            
            return result 
        else:
            return "Error: you must include the variable 'result' in the code you submit"
    except Exception as e:
        return e
        

def lambda_handler(event, context):

    print(event)
    
    ## Langchain wraps request and response in 'body', but Bedrock agents does not
    if 'body' in event:
        event = json.loads(event['body'])
    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])
    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }


    
    if function == 'run_python_code':
        
        
        cmd = "print('no valid python string entered')"
        for param in parameters:
            if param["name"] == "python_string":
                python_string = param["value"]

        output = run_python_code(python_string)
        responseBody =  {
            'TEXT': {
                "body": f"output: {output}"
            }
        }
    else:
        responseBody =  {
            'TEXT': {
                "body": f"output: Function called is invalid: {function}"
            }
        }
        
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }

    }
    
    ## Langchain wraps request and response in 'body', but Bedrock agents does not
    
    if isinstance( event['agent'] , dict):
        function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    else:
        function_response = {'body': {'response': action_response, 'messageVersion': event['messageVersion']}}
        
    
    print("Response: {}".format(function_response))

    return function_response


Overwriting lambda_function.py


## Step 2: Create Lambda function

In [4]:
lambda_iam_role = create_lambda_role(agent_name, dynamodb_table_name='') # Do not create DynamoDB table. If your Lambda requires DynamoDB, provide a table name
lambda_function_name = f'{agent_name}-lambda'
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)

## Step 3: Test model invoke and Lambda invoke

#### Test Model invoke separately

In [5]:
from langchain_aws import ChatBedrock, BedrockLLM

In [6]:
import boto3
client = boto3.client("bedrock")
model_list = [s['modelId'] for s in client.list_foundation_models()['modelSummaries']]

In [7]:
import ipywidgets

agent_llm_selector = ipywidgets.Dropdown(
    options=[(s,s) for s in model_list]
)

agent_llm_selector

Dropdown(options=(('amazon.titan-tg1-large', 'amazon.titan-tg1-large'), ('amazon.titan-image-generator-v1:0', …

In [8]:
llm = ChatBedrock(model_id=agent_llm_selector.value)

In [9]:
llm.invoke(input="Hello, ")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'usage': {'prompt_tokens': 10, 'completion_tokens': 12, 'total_tokens': 22}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 10, 'completion_tokens': 12, 'total_tokens': 22}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-d52a80c3-367b-40a5-a8e0-05ed04b88978-0')

#### Test Lambda invoke separately:

We will be mocking the test event that is required when using Amazon Bedrock Agents

In [10]:
import json 
lambda_client = boto3.client('lambda')

test_event = {
  "agent": "1234",
  "actionGroup": "5678",
  "messageVersion":"9101112",
  "function": "run_python_code",
  "parameters": [
    {
      "name":"python_string",
      "value":"result = 2 + 2"
    }
  ]
}

res = lambda_client.invoke(
            FunctionName=lambda_function_name,
            InvocationType="RequestResponse",
            Payload=json.dumps(test_event),
        )
            


In [11]:
json.dumps(test_event)

'{"agent": "1234", "actionGroup": "5678", "messageVersion": "9101112", "function": "run_python_code", "parameters": [{"name": "python_string", "value": "result = 2 + 2"}]}'

In [12]:
payload_stream = res["Payload"]
payload_string = payload_stream.read().decode("utf-8")
answer = json.loads(payload_string)["body"]
answer

{'response': {'actionGroup': '5678',
  'function': 'run_python_code',
  'functionResponse': {'responseBody': {'TEXT': {'body': 'output: 4'}}}},
 'messageVersion': '9101112'}

## Step 4: Test on Langchain

In [39]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(
    ["awslambda"],
    awslambda_tool_name="run_python_code",
    awslambda_tool_description="""Runs python code and returns result. Pass in input that has the following keys as a json 1. 'agent' (random id); 2. 'actionGroup' (random id); 3. 'function':'run_python_code'; 4. 'parameters' with the code to be run, where parameters is a list of nested json with parameter 'name' and 'value' keys; here the only required param key is 'python_string' (remember you must use the `result` variable when calcualting the final answer); 5. 'messageVersion' (random version) """,
    function_name=lambda_function['FunctionName'],
)

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

agent.run("""20% of 18924""")



> Entering new AgentExecutor chain...
Thought: To calculate 20% of 18924, I will need to use Python to perform the calculation.

Action: run_python_code
Action Input:
{
  "agent": "1234",
  "actionGroup": "5678",
  "function": "run_python_code",
  "parameters": [
    {
      "name": "python_string",
      "value": "result = 0.2 * 18924"
    }
  ],
  "messageVersion": "1.0"
}

Observation: Result: {'response': {'actionGroup': '5678', 'function': 'run_python_code', 'functionResponse': {'responseBody': {'TEXT': {'body': 'output: 3784.8'}}}}, 'messageVersion': '1.0'}
Thought:Question: 20% of 18924
Thought: To calculate 20% of 18924, I will need to use Python to perform the calculation.

Action: run_python_code
Action Input:
{
  "agent": "1234", 
  "actionGroup": "5678",
  "function": "run_python_code",
  "parameters": [
    {
      "name": "python_string",
      "value": "result = 0.2 * 18924"
    }
  ],
  "messageVersion": "1.0"
}

Observation: Result: {'response': {'actionGroup': '5678

'3784.8'

## Step 5: Deploy to Amazon Bedrock Agents

In [15]:
agent_name

'my-agent-us-east-17168459174841719336070-8890233'

In [16]:
suffix = f"{region}-{account_id}"

agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'role_{agent_name}'[:64]

agent_description = "Runs python code and returns result"

agent_instruction = tools[0].description

agent_action_group_description = "Action for " + agent_description

agent_action_group_name = suffix+"ActionGroup"

#### 5a. Create agent role

In [17]:
agent_role = create_agent_role(agent_name, agent_llm_selector.value)

In [18]:
agent_role

{'Role': {'Path': '/',
  'RoleName': 'RoleForAgents_my-agent-us-east-17168459174841719336070-8890233',
  'RoleId': 'AROA2NZ2YUUWKADAG7ILB',
  'Arn': 'arn:aws:iam::716845917484:role/RoleForAgents_my-agent-us-east-17168459174841719336070-8890233',
  'CreateDate': datetime.datetime(2024, 6, 25, 17, 22, 7, tzinfo=tzlocal()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]}},
 'ResponseMetadata': {'RequestId': '409fcce0-62d5-4d63-8600-c978f2597b02',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 25 Jun 2024 17:22:07 GMT',
   'x-amzn-requestid': '409fcce0-62d5-4d63-8600-c978f2597b02',
   'content-type': 'text/xml',
   'content-length': '879'},
  'RetryAttempts': 0}}

#### 5b. Create agent

In [19]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_llm_selector.value,
    instruction=agent_instruction,
)

agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

The agent id is: QA7LSTI0EY


#### 5c. Create agent action group

In [20]:
agent_functions = [
    {
        'name': tools[0].name,
        'description': tools[0].description,
        'parameters': {
            "python_string": {
                "description": "Python code to run. Can be multi-step but must have a 'result' variable.",
                "required": True,
                "type": "string"
            }
        }
    },]

In [21]:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [22]:
agent_action_group_response['agentActionGroup']

{'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:716845917484:function:my-agent-us-east-17168459174841719336070-8890233-lambda'},
 'actionGroupId': 'X30LHCNK8L',
 'actionGroupName': 'us-east-1-716845917484ActionGroup',
 'actionGroupState': 'ENABLED',
 'agentId': 'QA7LSTI0EY',
 'agentVersion': 'DRAFT',
 'createdAt': datetime.datetime(2024, 6, 25, 17, 22, 26, 170212, tzinfo=tzlocal()),
 'description': 'Action for Runs python code and returns result',
 'functionSchema': {'functions': [{'description': "Runs python code and returns result. Pass in input that has the following keys as a json 1. 'agent' (random id); 2. 'actionGroup' (random id); 3. 'function':'run_python_code'; 4. 'parameters' with the code to be run, where parameters is a list of nested json with parameter 'name' and 'value' keys; here the only required param key is 'python_string' (remember you must use the `result` variable when calcualting the final answer); 5. 'messageVersion' (random version) ",
    'name': 

#### 5d. Allow Bedrock to call Lambda functions

In [23]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

{'ResponseMetadata': {'RequestId': 'f3bc7f50-a6ee-478b-be46-a0a9989f5a0d', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Tue, 25 Jun 2024 17:22:30 GMT', 'content-type': 'application/json', 'content-length': '394', 'connection': 'keep-alive', 'x-amzn-requestid': 'f3bc7f50-a6ee-478b-be46-a0a9989f5a0d'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"allow_bedrock_QA7LSTI0EY","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:716845917484:function:my-agent-us-east-17168459174841719336070-8890233-lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:716845917484:agent/QA7LSTI0EY"}}}'}


#### 5e. Prepare the agent

In [24]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': 'a61e93c8-0246-4243-a9fb-7af6ed6301d2', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Tue, 25 Jun 2024 17:22:34 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'a61e93c8-0246-4243-a9fb-7af6ed6301d2', 'x-amz-apigw-id': 'Z7xyIG44IAMEcDQ=', 'x-amzn-trace-id': 'Root=1-667afcda-3ffb87fc79ec926b28b2cd1d'}, 'RetryAttempts': 0}, 'agentId': 'QA7LSTI0EY', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 6, 25, 17, 22, 34, 249747, tzinfo=tzlocal())}


#### 5f. Invoke the migrated agent

In [40]:
%%time
session_id:str = str(uuid.uuid1())
alias_id = 'TSTALIASID'
query = """20% of 18924"""
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

20% of 18924 is 3784.8
CPU times: user 14.4 ms, sys: 0 ns, total: 14.4 ms
Wall time: 9.42 s


## Step 6: Cleanup!

In [41]:
from agent import delete_agent_roles_and_policies, clean_up_resources

try:
    clean_up_resources(
    '', lambda_function, lambda_function_name, agent_action_group_response, agent_functions, 
    agent_id, '', alias_id)
    
    delete_agent_roles_and_policies(agent_name, kb_policy_name)
except Exception as e:
    print(e)


Error deleting Agent resources: An error occurred (AccessDeniedException) when calling the DisassociateAgentKnowledgeBase operation: Unable to determine service/operation name to be authorized
Lambda function my-agent-us-east-17168459174841719336070-8890233-lambda has been deleted.
Error deleting table : Parameter validation failed:
Invalid length for parameter TableName, value: 0, valid min length: 1
name 'kb_policy_name' is not defined
